# Импортируем необходимые библиотеки

In [1]:
!pip uninstall cudf cuml dask-cudf ucxx accelerate albucore albumentations altair apache-beam \
               arviz bayesian-optimization beatrix-jupyterlab bigframes bokeh cesium chex conda \
               dask-expr dataproc-jupyter-plugin datasets distributed fastapi featuretools flask \
               gcsfs google-ai-generativelanguage google-api-core google-cloud-aiplatform google-cloud-artifact-registry \
               google-cloud-automl google-cloud-bigquery google-cloud-bigquery-connection google-cloud-datastore \
               google-cloud-dlp google-cloud-functions google-cloud-iam google-cloud-language \
               google-cloud-monitoring google-cloud-pubsub google-cloud-resource-manager \
               google-cloud-spanner google-cloud-translate google-cloud-videointelligence \
               googleapis-common-protos grpc-google-iam-v1 jupyterlab jupyterlab-lsp \
               jupyterlab-server kaggle kaggle-environments kernels-mixer kfp \
               libpysal mizani multiprocess onnx pathos plotnine pyldavis \
               pyopenssl pytoolconfig rapids-dask-dependency \
               rmm s3fs tensorflow tensorflow-serving-api textblob tsfresh typeguard \
               woodwork xarray -q --yes

In [2]:
!wget "https://raw.githubusercontent.com/dllllb/pytorch-lifestream/refs/heads/main/requirements.txt" -O "requirements.txt"
!pip install --upgrade -r requirements.txt
!pip cache purge

--2025-01-18 21:07:15--  https://raw.githubusercontent.com/dllllb/pytorch-lifestream/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3585 (3.5K) [text/plain]
Saving to: 'requirements.txt'

requirements.txt    100%[===================>]   3.50K  --.-KB/s    in 0s      

2025-01-18 21:07:16 (49.4 MB/s) - 'requirements.txt' saved [3585/3585]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install pytorch-lifestream
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pytorch-lifestream: filename=pytorch_lifestream-0.6.0-py3-none-any.whl size=274640 sha256=fa1f185e70650d80abffcaaf3ee3ce6c84b7f6e0fc746b9d510a395a2ecc07fc
  Stored in directory: /root/.cache/pip/wheels/90/76/b4/0a944bc7c5a69201e4d757cc54886971117a2a581740e7f11d
Successfully built pytorch-lifestream
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 16.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.3 MB/s eta 0:00:00


In [4]:
# data preprocessing
import os
import numpy as np
import pandas as pd
import pickle

# misc
from tqdm import tqdm
from functools import partial

# logging
import comet_ml

# classical ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from catboost import CatBoostClassifier

# basic deep learning libs
import torch
import pytorch_lightning as pl
import torchmetrics

# ptls
from ptls.nn import TrxEncoder, RnnSeqEncoder, TransformerEncoder, GptEncoder, Head
from ptls.frames import PtlsDataModule
from ptls.frames.coles import CoLESModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames.cpc import CpcModule
from ptls.frames.cpc import CpcDataset
from ptls.frames.gpt import GptDataset
from ptls.frames.supervised import SeqToTargetDataset, SequenceToTarget
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.datasets import inference_data_loader
from ptls.frames.inference_module import InferenceModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.utils import collate_feature_dict
from ptls.frames.inference_module import InferenceModule

# Supervised-кейс. Бустинг

**Препроцессим данные:**

**Скачаем данные:**

In [32]:
path_data = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true"
data = pd.read_csv(path_data, compression="gzip")
data

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [33]:
path_target = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true"
target = pd.read_csv(path_target)
target

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3
...,...,...
29995,14303,1
29996,22301,2
29997,25731,0
29998,16820,3


**Делим таргет на трейн и на тест:**

In [34]:
target_train, target_test = train_test_split(target, test_size=0.1, stratify=target["bins"], random_state=42)

**Проводим препроцессинг транзакций (деление на трейн и на тест, подготовка данных под нужный формат):**

In [35]:
trx_data_train = pd.merge(data, target_train["client_id"], on="client_id", how="inner")
trx_data_test = pd.merge(data, target_test["client_id"], on="client_id", how="inner")

**Дополнительно: убираем все транзакции с редкими MCC-кодами:**

In [36]:
def filter_rows_by_MCC(data):
    MCC_top100 = data["small_group"].value_counts()[:100].index.to_list()
    data = data[data["small_group"].isin(MCC_top100)].reset_index(drop=True)
    return data

In [37]:
trx_data_train = filter_rows_by_MCC(trx_data_train)
trx_data_test = filter_rows_by_MCC(trx_data_test)

**Дополнительно: предобработка числовых признаков: $x \longrightarrow \text{sign}(x) \cdot \log(1 + |x|)$**

In [38]:
def preprocess_num_features(data, features_list):
    for feature in features_list:
        data[feature] = np.sign(data[feature]) * np.log1p(np.abs(data[feature]))
    return data

In [39]:
trx_data_train = preprocess_num_features(data=trx_data_train, features_list=["amount_rur"])
trx_data_test = preprocess_num_features(data=trx_data_test, features_list=["amount_rur"])

In [40]:
preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="trans_date",
    event_time_transformation="none",
    cols_category=["small_group"],
    cols_numerical=["amount_rur"],
    return_records=False,
)

In [41]:
trx_data_train = preprocessor.fit_transform(trx_data_train)
trx_data_test = preprocessor.transform(trx_data_test)

In [42]:
trx_data_train.drop(columns=["trans_date"], inplace=True)
trx_data_test.drop(columns=["trans_date"], inplace=True)

In [43]:
target_train.sort_values(by="client_id", inplace=True)
target_test.sort_values(by="client_id", inplace=True)

In [44]:
trx_data_train.drop(columns=["client_id"], inplace=True)
trx_data_test.drop(columns=["client_id"], inplace=True)
target_train.drop(columns=["client_id"], inplace=True)
target_test.drop(columns=["client_id"], inplace=True)

In [45]:
trx_data_train.reset_index(inplace=True, drop=True)
trx_data_test.reset_index(inplace=True, drop=True)
target_train.reset_index(inplace=True, drop=True)
target_test.reset_index(inplace=True, drop=True)

In [46]:
max_seq_length = 0 

for idx, row in trx_data_train.iterrows():
    max_seq_length = max(max_seq_length, row["event_time"].shape[0])

for idx, row in trx_data_test.iterrows():
    max_seq_length = max(max_seq_length, row["event_time"].shape[0])

print("Max Sequence Length:", max_seq_length)

Max Sequence Length: 1150


In [47]:
#columns = ["event_time", "small_group", "amount_rur"]

columns = ["small_group", "amount_rur"]

new_tables_columns = []

for i in range(1, max_seq_length + 1):
    for col in columns:
        new_tables_columns.append(col + "_" + str(i))

In [50]:
new_train_table_contents = []

for idx, row in tqdm(trx_data_train.iterrows()):
    new_row = []
    trans_date = row["event_time"]
    small_group = row["small_group"]
    amount = row["amount_rur"]
    for j in range(max_seq_length):
        if j < trans_date.shape[0]:
            #new_row.append(trans_date[j].item())
            new_row.append(small_group[j].item())
            new_row.append(amount[j].item())
        else:
            #new_row.append(-1)
            new_row.append(-1)
            new_row.append(0.)
    new_train_table_contents.append(new_row)

trx_data_train = pd.DataFrame(data=new_train_table_contents, columns=new_tables_columns)
new_train_table_contents = []

27000it [02:09, 208.86it/s]


In [52]:
new_test_table_contents = []

for idx, row in tqdm(trx_data_test.iterrows()):
    new_row = []
    trans_date = row["event_time"]
    small_group = row["small_group"]
    amount = row["amount_rur"]
    for j in range(max_seq_length):
        if j < trans_date.shape[0]:
            #new_row.append(trans_date[j].item())
            new_row.append(int(small_group[j].item()))
            new_row.append(amount[j].item())
        else:
            #new_row.append(-1)
            new_row.append(-1)
            new_row.append(0.)
    new_test_table_contents.append(new_row)

trx_data_test = pd.DataFrame(data=new_test_table_contents, columns=new_tables_columns)
new_test_table_contents = []

3000it [00:15, 199.59it/s]


In [54]:
cat_features = []

# for idx, feature in enumerate(new_tables_columns):
#     if idx % 3 == 0 or idx % 3 == 1:
#         cat_features.append(feature)

for idx, feature in enumerate(new_tables_columns):
    if idx % 2 == 0:
        cat_features.append(feature)

**Наконец, обучаем бустинг!**

In [56]:
clf = CatBoostClassifier(loss_function='MultiClass', cat_features=cat_features, task_type="GPU", devices='0')

clf.fit(trx_data_train, target_train, plot_file="catboost_log.html")

Learning rate set to 0.12714
0:	learn: 1.3660929	total: 60.3ms	remaining: 1m
1:	learn: 1.3484792	total: 118ms	remaining: 59s
2:	learn: 1.3328257	total: 171ms	remaining: 56.8s
3:	learn: 1.3190252	total: 223ms	remaining: 55.6s
4:	learn: 1.3068568	total: 275ms	remaining: 54.7s
5:	learn: 1.2959951	total: 325ms	remaining: 53.9s
6:	learn: 1.2860775	total: 376ms	remaining: 53.4s
7:	learn: 1.2772826	total: 427ms	remaining: 52.9s
8:	learn: 1.2686412	total: 476ms	remaining: 52.5s
9:	learn: 1.2609533	total: 527ms	remaining: 52.2s
10:	learn: 1.2540843	total: 577ms	remaining: 51.8s
11:	learn: 1.2476824	total: 625ms	remaining: 51.5s
12:	learn: 1.2412046	total: 674ms	remaining: 51.2s
13:	learn: 1.2356065	total: 723ms	remaining: 50.9s
14:	learn: 1.2304392	total: 772ms	remaining: 50.7s
15:	learn: 1.2246597	total: 820ms	remaining: 50.5s
16:	learn: 1.2192698	total: 869ms	remaining: 50.2s
17:	learn: 1.2144076	total: 917ms	remaining: 50s
18:	learn: 1.2088499	total: 966ms	remaining: 49.9s
19:	learn: 1.20392

In [57]:
test_pred = clf.predict(trx_data_test)
test_proba = clf.predict_proba(trx_data_test)

**Посчитаем метрики:**

In [5]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [59]:
print("CatBoost Accuracy:", accuracy_score(target_test["bins"], test_pred))
print("CatBoost ROC-AUC:", roc_auc_score(target_test["bins"], test_proba, average="weighted", multi_class="ovr"))

CatBoost Accuracy: 0.48133333333333334
CatBoost ROC-AUC: 0.7520544144726738


- "Flattened" Sequences + CatBoost (With Time Features):
  - `Accuracy: 0.5083333333333333`  
  - `ROC-AUC: 0.7634778739219664`

- "Flattened" Sequences + CatBoost (Without Time Features):
  - `Accuracy: 0.48133333333333334`  
  - `ROC-AUC: 0.7520544144726738`

- **Доп. препроцессинг данных (избавляемся от выбросов по MCC, переходим к log-шкале по `amount`) заведомо немного улучшает качество => будем его использовать.**

- **Без временных признаков качество стало хуже => их надо использовать в решении.**

---

# Supervised-кейс. GRU.

**Препроцессим данные:**

**Скачаем данные:**

In [123]:
path_data = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true"
data = pd.read_csv(path_data, compression="gzip")
data

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [124]:
path_target = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true"
target = pd.read_csv(path_target)
target

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3
...,...,...
29995,14303,1
29996,22301,2
29997,25731,0
29998,16820,3


**Делим таргет на трейн и на тест (поправить пайплайн с учётом этого факта):**

In [125]:
target_train, target_test = train_test_split(target, test_size=0.1, stratify=target["bins"], random_state=42)

In [126]:
trx_data_train = pd.merge(data, target_train["client_id"], on="client_id", how="inner")
trx_data_test = pd.merge(data, target_test["client_id"], on="client_id", how="inner")

**Препроцессинг:**

In [127]:
preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="trans_date",
    event_time_transformation="none",
    cols_category=["small_group"],
    cols_numerical=["amount_rur"],
    return_records=False,
)

In [128]:
data_train = preprocessor.fit_transform(trx_data_train)
data_test = preprocessor.transform(trx_data_test)

In [129]:
data_train = pd.merge(data_train, target_train, on="client_id")
data_test = pd.merge(data_test, target_test, on="client_id")

In [130]:
data_train.rename(columns={"bins": "target"}, inplace=True)
data_test.rename(columns={"bins": "target"}, inplace=True)

In [131]:
data_train = data_train.to_dict(orient="records")
data_test = data_test.to_dict(orient="records")

In [132]:
data_train = MemoryMapDataset(data_train)
data_test = MemoryMapDataset(data_test)

**Создаём DataLoader:**

In [133]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [134]:
seed_everything(0)

In [135]:
data = PtlsDataModule(
    train_data=SeqToTargetDataset(data_train, target_col_name="target", target_dtype=torch.long),
    valid_data=SeqToTargetDataset(data_test, target_col_name="target", target_dtype=torch.long),
    train_batch_size=128,
    valid_batch_size=128,
    train_num_workers=4
)

**Модель (GRU):**

In [136]:
N_EPOCHS = 20

In [137]:
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(
        embeddings={
            "small_group": {"in": 250, "out": 16},
            "trans_date": {"in": 800, "out": 16}
        },
        numeric_values={
            "amount_rur": "log",
        },
        embeddings_noise=0.003,
    ),
    hidden_size=256,
    is_reduce_sequence=True
)

In [138]:
gru = SequenceToTarget(
    seq_encoder=seq_encoder,
    head=Head(input_size=seq_encoder.embedding_size, objective="classification", num_classes=4, hidden_layers_sizes=[512]),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(task="multiclass", num_classes=4),
    optimizer_partial=partial(torch.optim.Adam, lr=3e-3, weight_decay=5e-4),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.CosineAnnealingLR, T_max=N_EPOCHS, eta_min=5e-5)
)

**Обучение:**

In [139]:
comet_ml.login()

In [140]:
from pytorch_lightning.loggers import CometLogger

In [141]:
logger = CometLogger(project_name="EvS_SSL", experiment_name="supervised_baseline_GRU")

trainer = pl.Trainer(
    logger=logger,
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True
)

In [142]:
trainer.fit(gru, data)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Failing to collect the installed pip packages
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/evs-ssl/aee4b4dde50940d2900824db8d2d63be

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : supervised_baseline_GRU
COMET INFO:     url                   : https://www.comet.com/askoro/evs-ssl/aee4b4dde50940d2900824db8d2d63be
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [506]                    : (0.6670958995819092, 1.3880764245986938)
COMET INFO:     seq_len [84]                  : (858.6484375, 902.578125)
COMET INFO:     val_loss [20]                 : (0.8775425553321838, 1.1176246404647827)
COMET INFO:     valid/MulticlassAccuracy [20] : (0.4816666543483734, 0.6050000190734863)
COMET INFO:     y [84]                        : (1.28125, 1.703125)
COMET INFO:   Others:
COMET INFO:     Name : supervised_baseline_GRU
COME

In [143]:
torch.save(gru.state_dict(), "supervised_gru_with_2layered_MLP.pt")

In [144]:
print(trainer.logged_metrics)

{'loss': tensor(0.8209), 'seq_len': tensor(879.2501), 'y': tensor(1.4583), 'val_loss': tensor(0.8794), 'valid/MulticlassAccuracy': tensor(0.6050)}


---

**Измерим качество на тесте:**

**Используем энкодер + MLP:**

In [156]:
test_loader = torch.utils.data.DataLoader(
    dataset=data_test,
    collate_fn=collate_feature_dict,
    shuffle=False,
    batch_size=128,
    num_workers=0,
)

In [157]:
model = InferenceModule(
    torch.nn.Sequential(
        gru,
        torch.nn.Softmax(dim=1),
    ),
    model_out_name="prob",
)

model.eval()

InferenceModule(
  (model): Sequential(
    (0): SequenceToTarget(
      (seq_encoder): RnnSeqEncoder(
        (trx_encoder): TrxEncoder(
          (embeddings): ModuleDict(
            (small_group): NoisyEmbedding(
              250, 16, padding_idx=0
              (dropout): Dropout(p=0, inplace=False)
            )
            (trans_date): NoisyEmbedding(
              800, 16, padding_idx=0
              (dropout): Dropout(p=0, inplace=False)
            )
          )
          (custom_embeddings): ModuleDict(
            (amount_rur): LogScaler()
          )
          (custom_embedding_batch_norm): RBatchNorm(
            (bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (seq_encoder): RnnEncoder(
          (rnn): GRU(33, 256, batch_first=True)
          (reducer): LastStepEncoder()
        )
      )
      (head): Head(
        (model): Sequential(
          (0): Linear(in_features=256, out_features=512, bias=True)

In [158]:
pred = trainer.predict(model, test_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/evs-ssl/aee4b4dde50940d2900824db8d2d63be

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.



Predicting: |          | 0/? [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : supervised_baseline_GRU
COMET INFO:     url                   : https://www.comet.com/askoro/evs-ssl/aee4b4dde50940d2900824db8d2d63be
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : supervised_baseline_GRU
COMET INFO:   Parameters:
COMET INFO:     test_batch_size   : None
COMET INFO:     test_drop_last    : False
COMET INFO:     test_num_workers  : None
COMET INFO:     train_batch_size  : 128
COMET INFO:     train_drop_last   : False
COMET INFO:     train_num_workers : 4
COMET INFO:     valid_batch_size  : 128
COMET INFO:     valid_drop_last   : False
COMET INFO:     valid_num_workers : No

In [159]:
pred = pd.concat(pred, axis=0)

In [160]:
pred

,client_id,target,prob_0000,prob_0001,prob_0002,prob_0003
0,14,0,0.222281,0.359911,0.081398,0.336410
1,19,1,0.013895,0.875499,0.001037,0.109569
2,64,3,0.094936,0.423387,0.003271,0.478405
3,78,2,0.061752,0.001211,0.933950,0.003087
4,105,2,0.381408,0.134703,0.039848,0.444042
...,...,...,...,...,...,...
51,49955,0,0.534354,0.048208,0.176690,0.240749
52,49963,3,0.593882,0.022068,0.221428,0.162621
53,49988,1,0.015238,0.844646,0.001356,0.138761
54,49995,2,0.124782,0.003852,0.861525,0.009840


In [161]:
y_pred = pred[[f"prob_{i:04d}" for i in range(4)]].values.argmax(axis=1)
y_pred

array([1, 1, 3, ..., 1, 2, 1])

In [162]:
pred

,client_id,target,prob_0000,prob_0001,prob_0002,prob_0003
0,14,0,0.222281,0.359911,0.081398,0.336410
1,19,1,0.013895,0.875499,0.001037,0.109569
2,64,3,0.094936,0.423387,0.003271,0.478405
3,78,2,0.061752,0.001211,0.933950,0.003087
4,105,2,0.381408,0.134703,0.039848,0.444042
...,...,...,...,...,...,...
51,49955,0,0.534354,0.048208,0.176690,0.240749
52,49963,3,0.593882,0.022068,0.221428,0.162621
53,49988,1,0.015238,0.844646,0.001356,0.138761
54,49995,2,0.124782,0.003852,0.861525,0.009840


In [163]:
y_true = pred["target"].values
y_true

array([0, 1, 3, ..., 1, 2, 1])

In [164]:
y_proba = pred[[f"prob_{i:04d}" for i in range(4)]].values
y_proba

array([[0.22228114, 0.359911  , 0.08139779, 0.33641008],
       [0.01389456, 0.8754993 , 0.00103746, 0.10956874],
       [0.09493641, 0.42338708, 0.00327115, 0.4784054 ],
       ...,
       [0.01523766, 0.8446458 , 0.00135595, 0.13876054],
       [0.1247824 , 0.00385215, 0.8615252 , 0.00984028],
       [0.05888796, 0.7175184 , 0.01006003, 0.21353361]], dtype=float32)

In [165]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [166]:
print("Accuracy:", accuracy_score(y_true, y_pred))
print("ROC-AUC:", roc_auc_score(y_true, y_proba, average="weighted", multi_class="ovr"))

Accuracy: 0.605
ROC-AUC: 0.8459288394564334


GRU + 2layer MLP Head:
  - `Accuracy: 0.605`
  - `ROC-AUC:  0.8459288394564334`

# Self-Supervised Case. COLES, CPC, GPT.

**Данные:**

In [67]:
path_data = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true"
data = pd.read_csv(path_data, compression="gzip")
data

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [68]:
path_target = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true"
target = pd.read_csv(path_target)
target

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3
...,...,...
29995,14303,1
29996,22301,2
29997,25731,0
29998,16820,3


In [69]:
target_train, target_test = train_test_split(target, test_size=0.1, stratify=target["bins"], random_state=42)

In [70]:
trx_data_train = pd.merge(data, target_train["client_id"], on="client_id", how="inner")
trx_data_test = pd.merge(data, target_test["client_id"], on="client_id", how="inner")

---

**Квантизация непрерывных признаков (опциональный шаг, нужен только для GPT):**

In [71]:
def digitize(input_array: np.array, q_count: int = 1, bins: np.array = None):
    """Quantile-based discretization function.

    Parameters:
    -------
    input_array (np.array): Input array.
    q_count (int): Amount of quantiles. Used only if input parameter `bins` is None.
    bins (np.array):
        If None, then calculate bins as quantiles of input array,
        otherwise only apply bins to input_array. Default: None

    Returns
    -------
    out_array (np.array of ints): discretized input_array
    bins (np.array of floats):
        Returned only if input parameter `bins` is None.
    """

    if bins is None:
        return_bins = True
        bins = np.quantile(input_array, q=[i / q_count for i in range(1, q_count)], axis=0)
    else:
        return_bins = False

    out_array = np.digitize(input_array, bins)

    if return_bins:
        return out_array, bins
    else:
        return out_array

In [72]:
BINS_NUM = 128

In [73]:
numeric_features = ["amount_rur"]

for feat in numeric_features:
    trx_data_train[feat], bins = digitize(trx_data_train[feat], q_count=BINS_NUM)
    trx_data_test[feat] = digitize(trx_data_test[feat], bins=bins)

In [74]:
import gc

gc.collect()

2888

---

In [75]:
preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="trans_date",
    event_time_transformation="none",
    cols_category=["small_group"],
    cols_numerical=["amount_rur"],
    return_records=False,
)

In [76]:
data_train = preprocessor.fit_transform(trx_data_train)
data_test = preprocessor.transform(trx_data_test)

In [77]:
target_train.rename(columns={"bins": "target"}, inplace=True)
target_test.rename(columns={"bins": "target"}, inplace=True)
target_train.sort_values(by="client_id", inplace=True)
target_test.sort_values(by="client_id", inplace=True)
target_train = target_train["target"]
target_test = target_test["target"]
target_train.reset_index(drop=True, inplace=True)
target_test.reset_index(drop=True, inplace=True)

In [78]:
data_train = data_train.to_dict(orient="records")
data_test = data_test.to_dict(orient="records")

---

- **COLES:**

In [15]:
seed_everything(0)

**DataLoaders:**

In [16]:
data = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=data_train,
            i_filters=[SeqLenFilter(min_seq_len=25)],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=30,
            cnt_max=190,
        ),
    ),
    train_num_workers=4,
    train_batch_size=128,
    valid_data=ColesDataset(
        MemoryMapDataset(
            data=data_test,
            i_filters=[SeqLenFilter(min_seq_len=25)],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=30,
            cnt_max=190,
        ),
    ),
    valid_num_workers=4,
    valid_batch_size=128
)

**Модель:**

In [17]:
N_EPOCHS = 20

In [18]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={"amount_rur": "log"},
    embeddings={
        "trans_date": {"in": 800, "out": 16},
        "small_group": {"in": 250, "out": 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=512,
    type="gru",
)

coles = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=3e-3, weight_decay=5e-4),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.CosineAnnealingLR, T_max=N_EPOCHS, eta_min=5e-5)
)

**Обучение:**

In [105]:
logger = CometLogger(project_name="EvS_SSL", experiment_name="CoLES_Baseline")

trainer = pl.Trainer(
    logger=logger,
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True
)

In [106]:
trainer.fit(coles, data)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Failing to collect the installed pip packages
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/evs-ssl/0e67531ac9414bc1981a7976b01e0069

COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : CoLES_Baseline
COMET INFO:     url                   : https://www.comet.com/askoro/evs-ssl/0e67531ac9414bc1981a7976b01e0069
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [506]              : (153.493408203125, 1953.078369140625)
COMET INFO:     seq_len [84]            : (105.6421890258789, 115.37187957763672)
COMET INFO:     valid/recall_top_k [20] : (0.46438804268836975, 0.796305239200592)
COMET INFO:   Others:
COMET INFO:     Name : CoLES_Baseline
COMET INFO:   Parameters:
COMET INFO:     test_batch_size   : None
COMET INFO:     test_drop_last    : False
COMET INFO:     test_num_workers  : None
COMET INFO:     train_batch_size  : 

In [107]:
trainer.logged_metrics

{'loss': tensor(145.5555),
 'seq_len': tensor(112.2350),
 'valid/recall_top_k': tensor(0.7942)}

In [108]:
torch.save(seq_encoder.state_dict(), "coles_enc_baseline.pt")

**Измерим качество на тесте (catboost поверх эмбеддингов):**

In [19]:
# !wget "https://drive.google.com/uc?export=download&id=1Mn8o9IPT4Zzg3946orbw1MVZwpkrBoNb" -O "coles_enc_baseline.pt"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2025-01-18 21:18:00--  https://drive.google.com/uc?export=download&id=1Mn8o9IPT4Zzg3946orbw1MVZwpkrBoNb
Resolving drive.google.com (drive.google.com)... 74.125.197.138, 74.125.197.102, 74.125.197.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.197.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Mn8o9IPT4Zzg3946orbw1MVZwpkrBoNb&export=download [following]
--2025-01-18 21:18:01--  https://drive.usercontent.google.com/download?id=1Mn8o9IPT4Zzg3946orbw1MVZwpkrBoNb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.188.132, 2607:f8b0:400e:c1b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.188.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3435234 (3.3M) [application/octet-stream]
Saving to: 'coles_enc_baseline.pt'

coles_enc_baseline. 100%[===================>]   3.28M 

In [20]:
encoder = coles.seq_encoder

# state_dict = torch.load("./coles_enc_baseline.pt")
# encoder.load_state_dict(state_dict)

device = "cuda:0"

encoder.to(device)

RnnSeqEncoder(
  (trx_encoder): TrxEncoder(
    (embeddings): ModuleDict(
      (trans_date): NoisyEmbedding(
        800, 16, padding_idx=0
        (dropout): Dropout(p=0, inplace=False)
      )
      (small_group): NoisyEmbedding(
        250, 16, padding_idx=0
        (dropout): Dropout(p=0, inplace=False)
      )
    )
    (custom_embeddings): ModuleDict(
      (amount_rur): LogScaler()
    )
    (custom_embedding_batch_norm): RBatchNorm(
      (bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (seq_encoder): RnnEncoder(
    (rnn): GRU(33, 512, batch_first=True)
    (reducer): LastStepEncoder()
  )
)

In [21]:
from tqdm import tqdm

seed_everything(0)

In [22]:
train_loader = inference_data_loader(data_train, num_workers=0, batch_size=128)
encoder.eval()
train_embeds = None

for i, batch in tqdm(enumerate(train_loader)):
    train_embeds_batch = encoder(batch.to(device))
    if i == 0:
        train_embeds = train_embeds_batch.detach().cpu().numpy()
    else:
        train_embeds = np.concatenate([train_embeds, train_embeds_batch.detach().cpu().numpy()], axis=0)
    
train_embeds

211it [00:23,  8.84it/s]


array([[ 0.40693808, -0.03753442,  0.74707246, ...,  0.0813555 ,
        -0.012108  , -0.4044453 ],
       [ 0.4095803 , -0.14109942,  0.87563264, ...,  0.00762153,
        -0.02665744,  0.08812606],
       [ 0.12658368, -0.05330365,  0.9316243 , ...,  0.02486727,
         0.00419268,  0.36441287],
       ...,
       [ 0.29617146, -0.14900862,  0.8076583 , ...,  0.1061729 ,
        -0.0047957 ,  0.14762385],
       [ 0.29201204, -0.08790259,  0.88752574, ...,  0.06656656,
        -0.0122736 , -0.8736355 ],
       [ 0.41544574, -0.3685151 ,  0.89880055, ...,  0.02250025,
        -0.0172288 , -0.21932535]], dtype=float32)

In [23]:
test_loader = inference_data_loader(data_test, num_workers=0, batch_size=128)
encoder.eval()
test_embeds = None

for i, batch in tqdm(enumerate(test_loader)):
    test_embeds_batch = encoder(batch.to(device))
    if i == 0:
        test_embeds = test_embeds_batch.detach().cpu().numpy()
    else:
        test_embeds = np.concatenate([test_embeds, test_embeds_batch.detach().cpu().numpy()], axis=0)
    
test_embeds

24it [00:02,  9.75it/s]


array([[ 0.4158509 , -0.15883715,  0.8926075 , ...,  0.05866282,
        -0.02485047,  0.4353296 ],
       [ 0.39960513, -0.06287923,  0.7960687 , ...,  0.0300503 ,
         0.00139666, -0.15503085],
       [ 0.05918387,  0.19842322,  0.8042084 , ...,  0.07361803,
         0.02141733, -0.38110086],
       ...,
       [ 0.3005669 , -0.5513453 ,  0.95127654, ...,  0.08756777,
        -0.04308328, -0.5152756 ],
       [ 0.04145339, -0.58060974,  0.8544944 , ...,  0.05916871,
        -0.01381077,  0.41732895],
       [ 0.24465705,  0.15819754,  0.95152396, ...,  0.07722902,
        -0.00599548,  0.62486315]], dtype=float32)

In [24]:
clf = CatBoostClassifier(loss_function='MultiClass', task_type="GPU", devices='0')

clf.fit(train_embeds, target_train, plot_file="catboost_log.html")

Learning rate set to 0.12714
0:	learn: 1.2989920	total: 11.6s	remaining: 3h 13m 57s
1:	learn: 1.2365920	total: 11.7s	remaining: 1h 36m 58s
2:	learn: 1.1882102	total: 11.7s	remaining: 1h 4m 38s
3:	learn: 1.1485005	total: 11.7s	remaining: 48m 28s
4:	learn: 1.1161767	total: 11.7s	remaining: 38m 47s
5:	learn: 1.0882269	total: 11.7s	remaining: 32m 19s
6:	learn: 1.0650179	total: 11.7s	remaining: 27m 41s
7:	learn: 1.0445255	total: 11.7s	remaining: 24m 14s
8:	learn: 1.0265064	total: 11.7s	remaining: 21m 32s
9:	learn: 1.0126240	total: 11.7s	remaining: 19m 23s
10:	learn: 0.9995543	total: 11.8s	remaining: 17m 37s
11:	learn: 0.9880137	total: 11.8s	remaining: 16m 9s
12:	learn: 0.9776527	total: 11.8s	remaining: 14m 54s
13:	learn: 0.9681790	total: 11.8s	remaining: 13m 50s
14:	learn: 0.9593313	total: 11.8s	remaining: 12m 55s
15:	learn: 0.9520554	total: 11.8s	remaining: 12m 6s
16:	learn: 0.9458190	total: 11.8s	remaining: 11m 23s
17:	learn: 0.9395964	total: 11.8s	remaining: 10m 45s
18:	learn: 0.9332783	

In [25]:
test_pred = clf.predict(test_embeds)
test_proba = clf.predict_proba(test_embeds)

In [26]:
print("Accuracy:", accuracy_score(target_test, test_pred))
print("ROC-AUC:", roc_auc_score(target_test, test_proba, average="weighted", multi_class="ovr"))

Accuracy: 0.6063333333333333
ROC-AUC: 0.8485032660206542


- COLES embeds + Catboost:
  - `Accuracy: 0.6063333333333333`
  -  `ROC-AUC: 0.8485032660206542`

---

- **CPC modeling:**

In [55]:
seed_everything(0)

**DataLoaders:**

In [43]:
data = PtlsDataModule(
    train_data=CpcDataset(
        MemoryMapDataset(data=data_train),
        min_len=1000,
        max_len=1200
    ),
    train_num_workers=4,
    train_batch_size=64,
    valid_data=CpcDataset(
        MemoryMapDataset(data=data_test),
        min_len=1000,
        max_len=1200
    ),
    valid_num_workers=4,
    valid_batch_size=64
)

**Модель:**

In [44]:
N_EPOCHS = 15

In [56]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={"amount_rur": "log"},
    embeddings={
        "trans_date": {"in": 800, "out": 128},
        "small_group": {"in": 250, "out": 128}
    }
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=512,
    type="gru"
)

cpc = CpcModule(
    seq_encoder=seq_encoder,
    n_forward_steps=6,
    n_negatives=40,
    optimizer_partial=partial(torch.optim.Adam, lr=2e-3),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.5)
)

**Обучение:**

In [58]:
logger = CometLogger(project_name="EvS_SSL", experiment_name="CPC_modeling_baseline")

trainer = pl.Trainer(
    logger=logger,
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True
)

In [59]:
trainer.fit(cpc, data)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Failing to collect the installed pip packages
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/evs-ssl/bcbcd2861fae4e0fa05c59f050b765d1

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : CPC_modeling_baseline
COMET INFO:     url                   : https://www.comet.com/askoro/evs-ssl/bcbcd2861fae4e0fa05c59f050b765d1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [380]              : (0.8411377668380737, 3.7676494121551514)
COMET INFO:     seq_len [63]            : (851.359375, 898.578125)
COMET INFO:     valid/cpc_accuracy [15] : (0.7607796788215637, 0.9146100878715515)
COMET INFO:   Others:
COMET INFO:     Name : CPC_modeling_baseline
COMET INFO:   Parameters:
COMET INFO:     test_batch_size   : None
COMET INFO:     test_drop_last    : False
COMET INFO:     test_num_workers  : None
COMET INFO:     train_batch_size  

In [60]:
trainer.logged_metrics

{'loss': tensor(0.9963),
 'seq_len': tensor(876.1833),
 'valid/cpc_accuracy': tensor(0.9146)}

In [61]:
torch.save(seq_encoder.state_dict(), "cpc_enc_baseline.pt")

**Измерим качество на тесте (catboost поверх эмбеддингов):**

In [46]:
# !wget "https://drive.google.com/uc?export=download&id=11j6QgNsdOSTK-GRaAJLKObDW7ehS_aqK" -O "cpc_enc_baseline_higher_trx_dim.pt"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



--2025-01-18 21:38:11--  https://drive.google.com/uc?export=download&id=11j6QgNsdOSTK-GRaAJLKObDW7ehS_aqK
Resolving drive.google.com (drive.google.com)... 74.125.197.113, 74.125.197.101, 74.125.197.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.197.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=11j6QgNsdOSTK-GRaAJLKObDW7ehS_aqK&export=download [following]
--2025-01-18 21:38:12--  https://drive.usercontent.google.com/download?id=11j6QgNsdOSTK-GRaAJLKObDW7ehS_aqK&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.188.132, 2607:f8b0:400e:c1b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.188.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281858 (5.0M) [application/octet-stream]
Saving to: 'cpc_enc_baseline_higher_trx_dim.pt'

cpc_enc_baseline_hi 100%[=================

In [57]:
encoder = cpc.seq_encoder

# state_dict = torch.load("./cpc_enc_baseline_higher_trx_dim.pt")
# encoder.load_state_dict(state_dict)

device = "cuda:0"

encoder.to(device)

RnnSeqEncoder(
  (trx_encoder): TrxEncoder(
    (embeddings): ModuleDict(
      (trans_date): NoisyEmbedding(
        800, 128, padding_idx=0
        (dropout): Dropout(p=0, inplace=False)
      )
      (small_group): NoisyEmbedding(
        250, 128, padding_idx=0
        (dropout): Dropout(p=0, inplace=False)
      )
    )
    (custom_embeddings): ModuleDict(
      (amount_rur): LogScaler()
    )
    (custom_embedding_batch_norm): RBatchNorm(
      (bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (seq_encoder): RnnEncoder(
    (rnn): GRU(257, 512, batch_first=True)
    (reducer): LastStepEncoder()
  )
)

In [58]:
encoder.seq_encoder.is_reduce_sequence = True

In [59]:
from tqdm import tqdm

seed_everything(0)

In [50]:
train_loader = inference_data_loader(data_train, num_workers=0, batch_size=128)
encoder.eval()
train_embeds = None

for i, batch in tqdm(enumerate(train_loader)):
    train_embeds_batch = encoder(batch.to(device))
    if i == 0:
        train_embeds = train_embeds_batch.detach().cpu().numpy()
    else:
        train_embeds = np.concatenate([train_embeds, train_embeds_batch.detach().cpu().numpy()], axis=0)
    
train_embeds

211it [00:26,  7.87it/s]


array([[ 0.10740659, -0.31269857,  0.27584156, ..., -0.16188024,
         0.14214373, -0.16249156],
       [ 0.18475403,  0.04266822, -0.04595035, ..., -0.09261759,
         0.13259636, -0.13119616],
       [ 0.02973818, -0.41895607, -0.20449215, ..., -0.03822838,
        -0.12994064, -0.04135744],
       ...,
       [-0.08190808,  0.06189765,  0.38444516, ...,  0.00534426,
        -0.41809323, -0.18285802],
       [-0.01138211,  0.01361396, -0.2003721 , ..., -0.03389682,
        -0.26725724, -0.04146025],
       [ 0.10472313,  0.08043359, -0.15558885, ..., -0.08148728,
         0.10476687, -0.12173226]], dtype=float32)

In [61]:
test_loader = inference_data_loader(data_test, num_workers=0, batch_size=128)
encoder.eval()
test_embeds = None

for i, batch in tqdm(enumerate(test_loader)):
    test_embeds_batch = encoder(batch.to(device))
    if i == 0:
        test_embeds = test_embeds_batch.detach().cpu().numpy()
    else:
        test_embeds = np.concatenate([test_embeds, test_embeds_batch.detach().cpu().numpy()], axis=0)
    
test_embeds

24it [00:02,  8.52it/s]


array([[-0.06690586,  0.10264479, -0.4164489 , ...,  0.06194592,
         0.03693053, -0.05470861],
       [ 0.03215792, -0.39735085, -0.16195466, ..., -0.08005238,
        -0.08197732, -0.125411  ],
       [-0.07211616, -0.12515262, -0.2479379 , ..., -0.09862465,
        -0.10269979, -0.23215559],
       ...,
       [ 0.11553599,  0.0303034 , -0.15588439, ..., -0.08699856,
        -0.04712782, -0.17401324],
       [ 0.03141807, -0.06604744, -0.13441944, ..., -0.06602852,
        -0.1329235 , -0.24725263],
       [ 0.02691221, -0.2152749 , -0.32269812, ..., -0.15868856,
         0.07448553, -0.22014065]], dtype=float32)

In [52]:
clf = CatBoostClassifier(loss_function='MultiClass', task_type="GPU", devices='0')

clf.fit(train_embeds, target_train, plot_file="catboost_log.html")

Learning rate set to 0.12714
0:	learn: 1.3188436	total: 14.1ms	remaining: 14.1s
1:	learn: 1.2684504	total: 24.6ms	remaining: 12.3s
2:	learn: 1.2291237	total: 34.9ms	remaining: 11.6s
3:	learn: 1.1973830	total: 45.4ms	remaining: 11.3s
4:	learn: 1.1721738	total: 55.5ms	remaining: 11.1s
5:	learn: 1.1491398	total: 65.8ms	remaining: 10.9s
6:	learn: 1.1286722	total: 76ms	remaining: 10.8s
7:	learn: 1.1122167	total: 86.2ms	remaining: 10.7s
8:	learn: 1.0967006	total: 96.5ms	remaining: 10.6s
9:	learn: 1.0830547	total: 107ms	remaining: 10.6s
10:	learn: 1.0724027	total: 117ms	remaining: 10.5s
11:	learn: 1.0617825	total: 127ms	remaining: 10.5s
12:	learn: 1.0521902	total: 138ms	remaining: 10.4s
13:	learn: 1.0435558	total: 148ms	remaining: 10.4s
14:	learn: 1.0364686	total: 158ms	remaining: 10.4s
15:	learn: 1.0293106	total: 169ms	remaining: 10.4s
16:	learn: 1.0221976	total: 179ms	remaining: 10.4s
17:	learn: 1.0156756	total: 190ms	remaining: 10.4s
18:	learn: 1.0102988	total: 201ms	remaining: 10.4s
19:	l

In [53]:
test_pred = clf.predict(test_embeds)
test_proba = clf.predict_proba(test_embeds)

In [54]:
print("Accuracy:", accuracy_score(target_test, test_pred))
print("ROC-AUC:", roc_auc_score(target_test, test_proba, average="weighted", multi_class="ovr"))

Accuracy: 0.5763333333333334
ROC-AUC: 0.8252403572367512


- CPC context embeds + Catboost (low dim of trx embeds: 16 + 16 + 1):
  - `Accuracy: 0.5296666666666666`
  - ` ROC-AUC: 0.7912313385598585`

\

- CPC context embeds + Catboost (higher dim of trx embeds: 128 + 128 + 1 (as in ptls-experiments)):
  - `Accuracy: 0.5763333333333334`
  - ` ROC-AUC: 0.8252403572367512`

**При использовании эмбеддингов транзакций более высокой размерности получаем лучшее качество => будем их использовать.**

---

- **GPT:**

In [115]:
seed_everything(0)

**DataLoaders:**

In [116]:
data = PtlsDataModule(
    train_data=GptDataset(
        MemoryMapDataset(data=data_train),
        min_len=1000,
        max_len=1200
    ),
    train_num_workers=4,
    train_batch_size=16,
    valid_data=GptDataset(
        MemoryMapDataset(data=data_test),
        min_len=1000,
        max_len=1200
    ),
    valid_num_workers=4,
    valid_batch_size=16
)

**Модель:**

In [117]:
from torchmetrics import MeanMetric
from typing import Tuple, Dict, List, Union
from torch import nn
from ptls.nn.seq_encoder.abs_seq_encoder import AbsSeqEncoder
from ptls.nn import PBL2Norm
from ptls.data_load.padded_batch import PaddedBatch


class MeanPooling(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pb: PaddedBatch):
        payload = pb.payload # (B, T, H)
        mask = pb.seq_len_mask.bool()
        pb_mean = payload.sum(dim=1) / mask.float().sum(dim=1, keepdim=True)
        return pb_mean


class StatPooling(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pb: PaddedBatch):
        payload = pb.payload # (B, T, H)
        mask = pb.seq_len_mask.bool()
        inf_mask = torch.zeros_like(mask, device=mask.device).float()
        inf_mask[~mask] = -torch.inf
        
        pb_mean = payload.sum(dim=1) / mask.float().sum(dim=1, keepdim=True)
        pb_max = torch.max(payload + inf_mask.unsqueeze(-1), dim=1)[0]
        pb_stat = torch.cat((pb_mean, pb_max), dim=1)
        return pb_stat


class GPTHead(torch.nn.Module):   
    def __init__(self, input_size, n_classes, hidden_size=64, drop_p=0.1):
        super().__init__()
        self.head = nn.Sequential(
            nn.Linear(input_size, hidden_size, bias=True),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(hidden_size, n_classes)
        )
    def forward(self, x):
        x = self.head(x)
        return x


class GptPretrainModule(pl.LightningModule):
    """GPT2 Language model

    Sequence transactions are encoded by `trx_encoder`.
    Then `seq_encoder` encodes the given sequence 
    (we actually use NN to modify sequence transactions representations,
    then (during inference) we calculate the mean of these encoded transactions to get the representation of the whole sequence).
    After this we use heads to predict the classes of features of the future transaction.

    Parameters
    ----------
    trx_encoder:
        Module for transform dict with feature sequences to sequence of transaction representations
    seq_encoder:
        Module for sequence processing. Generally this is transformer based encoder. Rnn is also possible
        Should work without sequence reduction
    head_hidden_size:
        Hidden size of heads for feature prediction
    seed_seq_len:
         Size of starting sequence without loss 
    total_steps:
        total_steps expected in OneCycle lr scheduler
    max_lr:
        max_lr of OneCycle lr scheduler
    weight_decay:
        weight_decay of Adam optimizer
    pct_start:
        % of total_steps when lr increase
    norm_predict:
        use l2 norm for transformer output or not
    """

    def __init__(self,
                 trx_encoder: torch.nn.Module,
                 seq_encoder: AbsSeqEncoder,
                 head_hidden_size: int = 64,
                 total_steps: int = 64000,
                 seed_seq_len: int = 16,
                 max_lr: float = 0.00005,
                 weight_decay: float = 0.0,
                 pct_start: float = 0.1,
                 norm_predict: bool = False
                 ):

        super().__init__()
        self.save_hyperparameters(ignore=['trx_encoder', 'seq_encoder'])

        self.trx_encoder = trx_encoder
        self._seq_encoder = seq_encoder
        self._seq_encoder.is_reduce_sequence = False

        self.head = nn.ModuleDict()
        for col_name, noisy_emb in self.trx_encoder.embeddings.items():
            self.head[col_name] = GPTHead(input_size=self._seq_encoder.embedding_size, hidden_size=head_hidden_size, n_classes=noisy_emb.num_embeddings)

        if self.hparams.norm_predict:
            self.fn_norm_predict = PBL2Norm()

        self.loss = nn.CrossEntropyLoss(ignore_index=0)

        self.train_gpt_loss = MeanMetric()
        self.valid_gpt_loss = MeanMetric()

    def forward(self, batch: PaddedBatch):
        z_trx = self.trx_encoder(batch) 
        out = self._seq_encoder(z_trx)
        if self.hparams.norm_predict:
            out = self.fn_norm_predict(out)
        return out

    def loss_gpt(self, logits, labels):
        loss = 0
        for col_name, head in self.head.items():
            y_pred = head(logits[:, self.hparams.seed_seq_len:-1, :])
            y_pred = y_pred.view(-1, y_pred.size(-1))

            y_true = labels[col_name][:, self.hparams.seed_seq_len+1:]
            y_true = torch.flatten(y_true.long())

            loss += self.loss(y_pred, y_true)
            
        return loss

    def training_step(self, batch, batch_idx):
        out = self.forward(batch)  # PB: B, T, H
        out = out.payload if isinstance(out, PaddedBatch) else out
        labels = batch.payload

        loss_gpt = self.loss_gpt(out, labels)
        self.train_gpt_loss(loss_gpt)
        self.log(f'loss', loss_gpt, sync_dist=True)
        return loss_gpt

    def validation_step(self, batch, batch_idx):
        out = self.forward(batch)  # PB: B, T, H
        out = out.payload if isinstance(out, PaddedBatch) else out
        labels = batch.payload

        loss_gpt = self.loss_gpt(out, labels)
        self.valid_gpt_loss(loss_gpt)

    def on_training_epoch_end(self):
        self.log('train loss (by epochs)', self.train_gpt_loss, prog_bar=True, logger=True, sync_dist=True, rank_zero_only=True)

    def on_validation_epoch_end(self):
        self.log('val loss (by epochs)', self.valid_gpt_loss, prog_bar=True, logger=True, sync_dist=True, rank_zero_only=True)

    def configure_optimizers(self):
        optim = torch.optim.NAdam(self.parameters(),
                                  lr=self.hparams.max_lr,
                                  weight_decay=self.hparams.weight_decay
                                 )
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optim,
            max_lr=self.hparams.max_lr,
            total_steps=self.hparams.total_steps,
            pct_start=self.hparams.pct_start,
            anneal_strategy='cos',
            cycle_momentum=False,
            div_factor=25.0,
            final_div_factor=10000.0,
            three_phase=False
        )
        
        scheduler = {'scheduler': scheduler, 'interval': 'step'}
        return [optim], [scheduler]
    
    @property
    def seq_encoder(self):
        return GPTInferenceModule(pretrained_model=self)


class GPTInferenceModule(torch.nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.model = pretrained_model
        self.model.is_reduce_sequence = False
        self.mean_pooling = MeanPooling()
        self.stat_pooling = StatPooling()

    def forward(self, batch, eval_strategy="mean"):
        z_trx = self.model.trx_encoder(batch)
        out = self.model._seq_encoder(z_trx)
        out = out if isinstance(out, PaddedBatch) else PaddedBatch(out, batch.seq_lens)

        if eval_strategy == "mean":
            out = self.mean_pooling(out)
        elif eval_strategy == "stat":
            out = self.stat_pooling(out)

        if self.model.hparams.norm_predict:
            out = out / (out.pow(2).sum(dim=-1, keepdim=True) + 1e-9).pow(0.5)
        return out

In [118]:
N_EPOCHS = 15

In [119]:
trx_encoder = TrxEncoder(
    embeddings_noise=0.003,
    embeddings={
        "trans_date": {"in": 730, "out": 64},
        "small_group": {"in": 204, "out": 64},
        "amount_rur": {"in": BINS_NUM, "out": 64}
    }
)

seq_encoder = GptEncoder(
    n_embd=trx_encoder.output_size,
    n_layer=6,
    n_head=6,
    n_inner=256,
    activation_function="gelu_new",
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1,
    n_positions=2048,
    use_positional_encoding=True,
    use_start_random_shift=True,
    is_reduce_sequence=False
)

gpt = GptPretrainModule(
    trx_encoder=trx_encoder,
    seq_encoder=seq_encoder,
    head_hidden_size=256,
    total_steps=(N_EPOCHS * 1688), # num_epochs * num_steps_per_epoch
    seed_seq_len=16,
    max_lr=1e-3,
    weight_decay=0.,
    pct_start=0.1,
    norm_predict=False
)

**Обучение:**

In [121]:
logger = CometLogger(project_name="EvS_SSL", experiment_name="GPT_modeling_baseline")

trainer = pl.Trainer(
    logger=logger,
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True
)

In [ ]:
trainer.fit(gpt, data)

In [47]:
trainer.logged_metrics

{'loss': tensor(7.7626), 'val loss (by epochs)': tensor(7.7473)}

In [101]:
encoder = gpt.seq_encoder

In [49]:
torch.save(encoder.state_dict(), "gpt_baseline.pt")

**Измерим качество на тесте (catboost поверх эмбеддингов):**

In [86]:
# !pip install gdown

In [102]:
# import gdown

# gdown.download("https://drive.google.com/uc?export=download&id=1YBstN7hpEIREo7zORmPoEZ_0NyBgfjm6", "gpt_baseline_NAdam.pt")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1YBstN7hpEIREo7zORmPoEZ_0NyBgfjm6
From (redirected): https://drive.google.com/uc?export=download&id=1YBstN7hpEIREo7zORmPoEZ_0NyBgfjm6&confirm=t&uuid=b0f44bc3-b84b-425c-968f-016e419987af
To: /kaggle/working/gpt_baseline_NAdam.pt
100%|██████████| 34.7M/34.7M [00:00<00:00, 83.5MB/s]


'gpt_baseline_NAdam.pt'

In [103]:
# state_dict = torch.load("./gpt_baseline_NAdam.pt")
# encoder.load_state_dict(state_dict)

device = "cuda:0"

encoder.to(device)

GPTInferenceModule(
  (model): GptPretrainModule(
    (trx_encoder): TrxEncoder(
      (embeddings): ModuleDict(
        (trans_date): NoisyEmbedding(
          730, 64, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (small_group): NoisyEmbedding(
          204, 64, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (amount_rur): NoisyEmbedding(
          128, 64, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
      )
      (custom_embeddings): ModuleDict()
    )
    (_seq_encoder): GptEncoder(
      (transf): GPT2Model(
        (wte): Embedding(4, 192)
        (wpe): Embedding(2048, 192)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-5): 6 x GPT2Block(
            (ln_1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Conv1D()
              (c_proj): Conv1D()
              (attn_dropout): Dropou

In [105]:
from tqdm import tqdm

seed_everything(0)

In [106]:
train_loader = inference_data_loader(data_train, num_workers=0, batch_size=8)
encoder.eval()
train_embeds = None

for i, batch in tqdm(enumerate(train_loader)):
    train_embeds_batch = encoder(batch.to(device), eval_strategy="mean")
    if i == 0:
        train_embeds = train_embeds_batch.detach().cpu().numpy()
    else:
        train_embeds = np.concatenate([train_embeds, train_embeds_batch.detach().cpu().numpy()], axis=0)
    
train_embeds

3375it [03:09, 17.85it/s]


array([[-0.3218786 ,  0.8942813 ,  0.18234766, ...,  0.41674083,
        -0.30047607, -1.0540302 ],
       [ 0.45865428,  1.1590631 ,  0.3696621 , ...,  1.1603724 ,
        -0.07634397,  0.19446537],
       [-0.49546695,  0.40549877,  0.14452296, ..., -0.52580696,
        -0.25774506,  0.89303625],
       ...,
       [ 0.00473687,  0.23853965, -0.55238575, ..., -0.32655022,
         0.7277668 , -0.14123408],
       [-0.03366534, -0.02078401,  0.0037288 , ..., -0.35077643,
        -0.2963082 ,  0.15920848],
       [ 0.08042946,  1.1210668 , -0.1188014 , ...,  0.07889172,
         0.6780111 ,  0.00702281]], dtype=float32)

In [107]:
test_loader = inference_data_loader(data_test, num_workers=0, batch_size=1)
encoder.eval()
test_embeds = None

for i, batch in tqdm(enumerate(test_loader)):
    test_embeds_batch = encoder(batch.to(device), eval_strategy="mean")
    if i == 0:
        test_embeds = test_embeds_batch.detach().cpu().numpy()
    else:
        test_embeds = np.concatenate([test_embeds, test_embeds_batch.detach().cpu().numpy()], axis=0)
    
test_embeds

3000it [00:22, 131.43it/s]


array([[-0.15779176, -0.02425155,  0.07104921, ..., -0.40337446,
         0.37797025, -0.57727844],
       [-0.02391498, -0.02910606,  0.11117481, ..., -0.6577504 ,
        -0.18194324, -0.15969224],
       [ 0.12826063,  0.12188778,  0.46610692, ..., -0.00399132,
         0.18010819,  0.39666843],
       ...,
       [ 0.11036664,  0.20503697, -0.03281356, ...,  0.5371503 ,
        -0.60937643,  0.22674431],
       [-0.0154385 ,  0.1866453 ,  0.14716536, ..., -0.7518635 ,
        -0.36799467,  0.21461183],
       [ 0.0334779 ,  0.10179763,  0.18172781, ..., -0.01951507,
         0.10009006, -0.34604466]], dtype=float32)

In [108]:
clf = CatBoostClassifier(loss_function='MultiClass', task_type="GPU", devices='0')

clf.fit(train_embeds, target_train, plot_file="catboost_log.html")

Learning rate set to 0.12714
0:	learn: 1.3183775	total: 9.68ms	remaining: 9.67s
1:	learn: 1.2678618	total: 16.9ms	remaining: 8.41s
2:	learn: 1.2272969	total: 24.2ms	remaining: 8.05s
3:	learn: 1.1936583	total: 31.6ms	remaining: 7.86s
4:	learn: 1.1654606	total: 38.5ms	remaining: 7.67s
5:	learn: 1.1415911	total: 45.7ms	remaining: 7.57s
6:	learn: 1.1191799	total: 52.8ms	remaining: 7.48s
7:	learn: 1.0997703	total: 59.9ms	remaining: 7.43s
8:	learn: 1.0824855	total: 67.1ms	remaining: 7.39s
9:	learn: 1.0670313	total: 74.4ms	remaining: 7.36s
10:	learn: 1.0537688	total: 81.7ms	remaining: 7.34s
11:	learn: 1.0420689	total: 88.8ms	remaining: 7.31s
12:	learn: 1.0314238	total: 96.5ms	remaining: 7.33s
13:	learn: 1.0209709	total: 104ms	remaining: 7.33s
14:	learn: 1.0115736	total: 111ms	remaining: 7.29s
15:	learn: 1.0028790	total: 118ms	remaining: 7.27s
16:	learn: 0.9949229	total: 125ms	remaining: 7.24s
17:	learn: 0.9872802	total: 132ms	remaining: 7.21s
18:	learn: 0.9801586	total: 139ms	remaining: 7.17s

In [109]:
test_pred = clf.predict(test_embeds)
test_proba = clf.predict_proba(test_embeds)

In [110]:
print("Accuracy:", accuracy_score(target_test, test_pred))
print("ROC-AUC:", roc_auc_score(target_test, test_proba, average="weighted", multi_class="ovr"))

Accuracy: 0.6246666666666667
ROC-AUC: 0.8523603311141598


- GPT embeds + Catboost (higher dim of trx embeds: 64 + 64 + 64):
  - `Accuracy: 0.6246666666666667`
  - `ROC-AUC: 0.8523603311141598`

**При использовании эмбеддингов транзакций более низкой размерности получаем качество хуже => будем использовать эмбеддинги транзакций более высокой размерности.**

- GPT embeds + Catboost, NAdam with OneCycleLR:
  - `Accuracy: 0.6246666666666667`
  - `ROC-AUC: 0.8523603311141598`

\\

- GPT embeds + Catboost, RAdam with CosineAnnealingLR:
  - `Accuracy: 0.608`
  - `ROC-AUC: 0.8510760556937854`

**При использовании `NAdam` с `OneCycleLR` получаем лучшее качество => будем использовать их.**

**Дополнительно:** конкатенация результатов MeanPooling и MaxPooling при инференсе демонстрирует худшие результаты по сравнению с чистым MeanPooling:

- GPT embeds + Catboost, only MeanPooling at inference:
    - `Accuracy: 0.6246666666666667`
    - `ROC-AUC: 0.8523603311141598`

\

- GPT embeds + Catboost, MeanPooling and MaxPooling concatenation at inference:
    - `Accuracy: 0.6133333333333333`
    - `ROC-AUC: 0.8509477995775881`

**Вывод:** будем использовать только MeanPooling на этапе инференса.

# Итоги.

| Method                  |    Accuracy   | ROC-AUC       |
|-------------------------|---------------|---------------|
| **Flattened Sequences** | 0.508         | 0.7635        |
| **GRU (+ MLP)**         | 0.605         | 0.8459        |
| **CoLES**               | 0.606         | 0.8485        |
| **CPC Modeling**        | 0.576         | 0.8252        |
| **GPT2**                | 0.625         | 0.8524        |